# Setup course and import data

In [ ]:
# leadership = CourseType.objects.get_or_create(
#     name="leadership",
#     name_fa="پیش‌وایی",
#     category=1
# )[0]
# relationship = CourseType.objects.get_or_create(
#     name="relationship",
#     name_fa="رابطه",
#     category=1
# )[0]
# pileh = CourseType.objects.get_or_create(
#     name="pileh",
#     name_fa="پیله",
#     category=1
# )[0]
# heart_of_matter = CourseType.objects.get_or_create(
#     name="heart of matter",
#     name_fa="جان کلام",
#     category=2
# )[0]
# accomplishment = CourseType.objects.get_or_create(
#     name="accomplishment",
#     name_fa="دستاورد",
#     category=2
# )[0]
# confront = CourseType.objects.get_or_create(
#     name="confront",
#     name_fa="مواجهه",
#     category=2
# )[0]
# transforming_year = CourseType.objects.get_or_create(
#     name="transforming year",
#     name_fa="تحویل سال",
#     category=2
# )[0]

# course_to_create = []
# course_to_create = course_to_create + [(leadership, i+1) for i in range(20)]
# course_to_create = course_to_create + [(relationship, i+1) for i in range(5)]
# course_to_create = course_to_create + [(pileh, i+1) for i in range(6)]
# course_to_create = course_to_create + [(heart_of_matter, i+1) for i in range(3)]
# course_to_create = course_to_create + [(accomplishment, i+1) for i in range(1)]
# course_to_create = course_to_create + [(confront, i+1) for i in range(3)]
# course_to_create = course_to_create + [(transforming_year, i+1) for i in range(3)]
# done_courses = Course.objects.bulk_create(
#     Course(
#         course_type=course_type,
#         number = i,
#         course_name=f"{course_type} {i}"
#     ) for course_type, i in course_to_create
# )

In [ ]:
from tqdm import tqdm
import pandas as pd
from commons.utils import convert_to_english_digit, normalize_phone

df = pd.read_excel("notebooks/course info.xlsx", sheet_name="تجربه‌گرها")
df = df.where(pd.notnull(df), None)

# df['normalized_phone'] = df['fixed phone'].apply(normalize_phone)
# good_phones = df['normalized_phone'].dropna().unique()

good_phone = []
bad_phone = []
for phone in df["fixed phone"]:
    if len(str(phone).strip()) == 11 and str(phone).startswith("09"):
        good_phone.append("+989" + phone[2:])
    else:
        bad_phone.append(phone)
good_phone = list(set(good_phone))
bad_phone = list(set(bad_phone))


def get_gender(gender):
    if gender in ["M", "m", "مرد"]:
        return 1
    elif gender in ["F", "f", "زن"]:
        return 2
    return None


def get_national_id(national_id):
    if not national_id:
        return None
    if len(str(national_id).strip()) == 8:
        return "00" + convert_to_english_digit(str(national_id)).strip()
    return convert_to_english_digit(str(national_id)).strip()


def get_status(status):
    if status in ["انتقال به دوره های بعد", "انصراف قبل از دوره", "غایب", "انصراف پیش از دوره"]:
        return 2, 3
    elif status in ["انصراف روز اول", "انتخاب نماندن"]:
        return 3, 3
    elif status in ["انصراف روز اول - غایب", "ترک دوره"]:
        return 4, 3
    elif status in ["تیم اجرایی"]:
        return 7, 3
    elif status in ["مهمان"]:
        return 8, 3
    elif status in ["شرکت کننده", "حاضر در دوره"]:
        return 9, 3
    elif status in ["ثبت‌نام", "در انتظار پرداخت"]:
        return 1, 3
    elif status in ["حاضر در دوره - انتظار پرداخت"]:
        return 9, 2
    elif status in ["حاضر در دوره - عدم سررسید"]:
        return 9, 1
    elif status in ["حاضر در دوره - تسویه"]:
        return 9, 3
    return 9, 3


users_dic = {user[1]: user for user in User.objects.all().values_list("id", "username", "first_name", "last_name", "phone_number")}
users_dup_check = {f"{u[2]}{u[3]}".replace(" ", ""): u for u in users_dic.values() if u[3] not in [None, ""]}

done_registrations = Registration.objects.values_list("user_id", "course_id")
registration_dic = {f"{reg[0]}-{reg[1]}": True for reg in done_registrations}
course_dic = {f"{course.course_type.name_fa}-{course.number}": course for course in Course.objects.select_related("course_type").all()}

bad_data = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    phone = normalize_phone(row["fixed phone"])
    first_name = row.get("نام", "")
    if first_name:
        first_name = first_name.strip()
    last_name = row.get("نام‌خانوادگی", "")
    if last_name:
        last_name = last_name.strip()
    telegram_id = row.get("تلگرام", None) or row.get("آی‌دی تلگرام", None) or ""
    email = row.get("ایمیل", "") or ""
    profession = row.get("حرفه", "")
    age = row.get("سن", None)
    gender = get_gender(row.get("جنسیت", None))
    national_id = get_national_id(row.get("کد ملی", "")) or ""
    english_first_name = row.get("Name", "") or ""
    english_last_name = row.get("Surname", "") or ""
    referer_name = row.get("معرف", None) or ""

    if phone is None and first_name is None and last_name is None:
        continue

    last_name = last_name or ""
    if national_id and len(national_id) > 10:
        print("Invalid national ID", index, phone, national_id)
        national_id = ""
    if phone is None:
        username = f"adding_user_all_course_{index}"
    else:
        username = phone
        if f"adding_user_all_course_{index}" in users_dic and users_dic[f"adding_user_all_course_{index}"][4] is None:
            User.objects.filter(username=f"adding_user_all_course_{index}").update(username=phone, phone_number=phone)
            users_dic[phone] = users_dic[f"adding_user_all_course_{index}"]

    if username in users_dic:
        user = users_dic[username]
        if user[2] != first_name or user[3] != last_name:
            User.objects.filter(username=username).update(first_name=first_name, last_name=last_name)
            bad_data.append((index, phone, user[2], user[3], first_name, last_name))
            continue

    elif users_dup_check.get(f"{first_name}{last_name}".replace(" ", ""), None) is not None:
        user = users_dup_check[f"{first_name}{last_name}".replace(" ", "")]
        u = User.objects.get(id=user[0])
        if user[4] is None:
            u.phone_number = phone
            u.username = username
            users_dic[username] = [u.id, username, first_name, last_name, phone]
            user = users_dic[username]
        u.telegram_id = u.telegram_id if u.telegram_id not in [None, ""] else telegram_id
        u.email = u.email if u.email not in [None, ""] else email
        u.profession = u.profession if u.profession not in [None, ""] else profession
        u.age = u.age if u.age not in [None, 0] else age
        u.gender = u.gender if u.gender not in [None, ""] else gender
        u.national_id = u.national_id if u.national_id not in [None, ""] else national_id
        u.english_first_name = u.english_first_name if u.english_first_name not in [None, ""] else english_first_name
        u.english_last_name = u.english_last_name if u.english_last_name not in [None, ""] else english_last_name
        u.referer_name = u.referer_name if u.referer_name not in [None, ""] else referer_name
        u.save()
    else:
        user = User.objects.create(
            username=username,
            phone_number=phone,
            first_name=first_name,
            last_name=last_name,
            telegram_id=telegram_id,
            email=email,
            profession=profession,
            age=age,
            gender=gender,
            english_first_name=english_first_name,
            english_last_name=english_last_name,
            referer_name=referer_name,
        )
        user = [user.id, user.username, user.first_name, user.last_name, user.phone_number]
        users_dup_check[f"{first_name}{last_name}".replace(" ", "")] = user
        users_dic[username] = user
    course_name = row["نام دوره"]
    course_number = int(row["شماره دوره"])
    course = course_dic.get(f"{course_name}-{course_number}", None)
    if not course:
        print(f"Course not found: {course_name} - {course_number}", index, phone)
        continue
    status, payment_status = get_status(row.get("وضعیت", None))
    if registration_dic.get(f"{user[0]}-{course.id}", None) is not None:
        continue
    try:
        reg = Registration.objects.create(
            user_id=user[0],
            course=course,
            status=status,
            payment_status=payment_status,
        )
        registration_dic[f"{user[0]}-{course.id}"] = True
    except:
        print("Error", phone, user[2], user[3], row.get("نام"))

print("Done", len(bad_data), "bad data entries")


User.objects.filter(referer_name="nan").update(referer_name="")
User.objects.filter(english_first_name="nan").update(english_first_name="")
User.objects.filter(english_last_name="nan").update(english_last_name="")
User.objects.filter(telegram_id="nan").update(telegram_id="")
User.objects.filter(email="nan").update(email="")
User.objects.filter(national_id="nan").update(national_id="")
User.objects.filter(profession="nan").update(profession="")

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 2048/2048 [00:06<00:00, 301.05it/s]


Done 10 bad data entries


0

# Setup support team

In [ ]:
import random
import string

support_team = [
    ["احمد", "محمدی لیری", "+989163918667"],
    ["حسین", "سلمان", "+989367759388"],
    ["رویا", "مرادی‌زاده", "+989903283583"],
    ["سیدمحمد", "سیارپور", "+989125215405"],
    ["مریم", "جاوید", "+989126261105"],
    ["شهاب", "آذرکسب", "+989939194985"],
    ["زهرا", "یارایی شهمیرزادی", "+989356815558"],
    ["امیر", "کرمانشاهانی", "+989011063032"],
    ["ائل‌نور", "نقی‌نژاد", "+989149151966"],
    ["محمود", "کاشف", "+989103000558"],
    ["مرضیه", "خدادادی", "+989012572179"],
    ["علی", "آرمیده", "+989932942418"],
    ["فرزانه", "فیروز", "+989195061290"],
    ["حمیدرضا", "حر", "+989367704992"],
    ["صدف", "علیدوست", "+989366314477"],
    ["فرزاد", "سلطان آبادی", "+989125936532"],
    ["مرجان", "ناقور", "+989122208940"],
    ["مهسا", "سبزی", "+989102722402"],
    ["امیرمحمد", "حریمی", "+989301411201"],
    ["دیبا", "یکتا", "+989140020678"],
]
data = []
for first_name, last_name, phone in support_team:
    if not User.objects.filter(phone_number=phone).exists():
        user = User.objects.create(
            phone_number=phone,
            username=phone,
            first_name=first_name,
            last_name=last_name,
            is_staff=True,
            is_superuser=False,
        )
        random_password = "".join(random.choices(string.ascii_letters + string.digits, k=10))
        user.set_password(random_password)
        data.append([first_name, last_name, phone, random_password])
        print(first_name, last_name, phone, random_password)
    elif User.objects.filter(phone_number=phone, is_staff=False).exists():
        user = User.objects.get(phone_number=phone)
        user.is_staff = True
        user.save()
        random_password = "".join(random.choices(string.ascii_letters, k=12))
        user.set_password(random_password)
        user.save()
        data.append([first_name, last_name, phone, random_password])
        print(first_name, last_name, phone, random_password)

from tqdm import tqdm

for d in tqdm(support_team):
    _, _, phone = d
    user = User.objects.get(phone_number=phone)
    user.groups.add(Group.objects.get(name="supporting"))
    user.save()

from django.contrib.auth.models import Group, Permission

code_names_support = [
    "view_crmlog",
    "add_crmlog",
    "change_crmlog",
    "view_crmuser",
    "change_crmuser",
    "view_registration",
    "change_registration",
    "view_user",
]
supporting_group, created = Group.objects.get_or_create(name="supporting")
for code_name in code_names_support:
    perm = Permission.objects.get(codename=code_name)
    supporting_group.permissions.add(perm)

# Assign called crmuser from file

In [ ]:
import pandas as pd
from tqdm import tqdm
import re
from commons.utils import convert_to_english_digit, normalize_phone


def extract_text(text, default=None):
    if text is None or str(text) in ["nan", "Nan", "NaN"]:
        return default
    return str(text)


staff_map = {
    "احمد": "+989163918667",
    "حسین": "+989367759388",
    "فرزاد": "+989125936532",
    "رویا": "+989903283583",
    "محمد سیارپور": "+989125215405",
    "ماری": "+989126261105",
    "شهاب": "+989939194985",
    "زهرا": "+989356815558",
    "امیر": "+989011063032",
    "ائل‌نور": "+989149151966",
    "محمود": "+989103000558",
    "مرضیه": "+989012572179",
    "علی": "+989932942418",
    "فرزانه": "+989195061290",
    "حمیدرضا حر": "+989367704992",
    "صدف علی‌دوست": "+989366314477",
    "مرجان": "+989122208940",
    "مهسا سبزی": "+989102722402",
    "امیرمحمد حریمی": "+989301411201",
    "دیبا": "+989140020678",
}
staff_map_user = {n: User.objects.get(phone_number=phone) for n, phone in staff_map.items()}


not_found = []
no_support = []
correct = 0
added = 0

dfs = [pd.read_excel("notebooks/call.xlsx", sheet_name=f"Team {i + 1}", dtype=str) for i in range(4)]
for df in dfs:
    df["phone"] = df["شماره تماس"].apply(normalize_phone)
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        phone = row["phone"]
        if phone is None or str(phone) in ["nan", "Nan", "NaN"]:
            continue
        if (
            row.get("هم‌یار") is None
            or str(row.get("هم‌یار")) in ["nan", "Nan", "NaN"]
            or staff_map_user.get(row.get("هم‌یار", "").strip()) is None
        ):
            print(row.get("هم‌یار"))
            no_support.append((index, phone, row.get("هم‌یار")))
            continue
        support = staff_map_user.get(row.get("هم‌یار", "").strip())
        if phone is not None and phone != "" and phone != "0" and phone != "0":
            if User.objects.filter(phone_number=phone).count() == 0:
                not_found.append((index, phone))
                continue
            correct += 1
            user = User.objects.get(phone_number=phone)
            crm = user.crm_user
            crm.supporting_user = support
            crm.save()
            answer = row.get("پاسخ داد؟")
            desc = extract_text(row.get("توضیحات"), "").strip()
            crs = extract_text(row.get("دوره‌هایی که بوده"), "")
            exp = extract_text(row.get("تجربه‌ش از دوره"), "")
            how = extract_text(row.get("حالش چطوره؟"), "")
            description = (((desc + "\n" + crs).strip() + "\n" + exp).strip() + "\n" + how).strip()
            if CrmLog.objects.filter(crm=crm, user=support, action=1).exists():
                continue
            if len(description) > 1 or str(answer) == "True":
                added += 1
                crm_log = CrmLog.objects.create(crm=crm, user=support, action=1, description=description)
print(len(not_found), "not found", len(no_support), "no support", added, "added", correct, "correct")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 999/999 [00:14<00:00, 68.56it/s]

0 not found 0 no support 0 added 156 correct


# Assign crm user to support

In [34]:
from users.models import CrmUser, User


support_team = list(User.objects.filter(is_staff=True, is_superuser=False))
ids = list(
    CrmUser.objects.filter(user__main_user__isnull=True, user__phone_number__isnull=False, supporting_user__isnull=True).values_list(
        "id", flat=True
    )
)
print(len(ids), "users without support", len(support_team), "support team members")

1742 users without support 20 support team members


In [ ]:
from courses.models import Registration
from users.models import CrmUser
import random
from tqdm import tqdm, trange

random.shuffle(ids)
crm_map = {}
n = len(support_team)
for idx, crm_id in enumerate(ids):
    if crm_map.get(idx % n, None) is None:
        crm_map[idx % n] = []
    else:
        crm_map[idx % n].append(crm_id)
for i in trange(n):
    CrmUser.objects.filter(id__in=crm_map[i]).update(supporting_user=support_team[i], status=1)

/tmp/ipykernel_67739/1212477816.py:1: DeprecationWarning: module 'sre_parse' is deprecated
  from sre_parse import State
  0%|                                                                                                              | 0/20 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 11.10it/s]


In [ ]:
import pandas as pd
from tqdm import tqdm
from jdatetime import datetime as jdatetime
from django.utils import timezone
from financials.models import Transaction


def get_account(account_name):
    if account_name == "کارت نیما":
        return blue
    if account_name == "پی‌پینگ نیما":
        return payping
    if account_name == "پی‌پینگ علیرضا":
        return payping_alireza
    if account_name == "پدیدار":
        return padidar


def get_course(course, course_number):
    if course == "پیش‌وایی":
        return Course.objects.get(course_type__name_fa="پیش‌وایی", number=course_number)
    if course == "رابطه":
        return Course.objects.get(course_type__name_fa="رابطه", number=course_number)
    if course == "مواجهه":
        return Course.objects.get(course_type__name_fa="مواجهه", number=course_number)
    if course == "جان‌کلام":
        return Course.objects.get(course_type__name_fa="جان کلام", number=course_number)
    if course == "پویش" and course_number == 1:
        return Course.objects.get(course_type__name_fa="دستاورد", number=course_number)
    if course == "پویش" and course_number == 2:
        return Course.objects.get(course_type__name_fa="تحویل سال", number=course_number)
    if course == "پیله":
        return Course.objects.get(course_type__name_fa="پیله", number=course_number)


def get_category(category):
    if category == "ثبت‌نام دوره":
        return 1
    if category == "قسط دوره":
        return 3
    if category == "هزینه دوره":
        return 2
    if category == "هزینه عملیاتی":
        return 5
    if category == "حقوق":
        return 6
    if category == "سرمایه گذاری":
        return 8
    if category == "وام":
        return 10
    return None

payping = FinancialAccount.objects.get(name="پی‌پینگ")
payping_alireza = FinancialAccount.objects.get(name="پی‌پینگ علیرضا")
blue = FinancialAccount.objects.get(name="کارت بلو نیما")
padidar = FinancialAccount.objects.get(name="پدیدار")

df = pd.read_excel("notebooks/Financials.xlsx", sheet_name="CostIncome", dtype={"date": str})

df = df[list(df.columns)[:9]]
df.columns = ["date", "number", "course", "course_number", "amount", "person", "account", "description", "category"]
# df = df.iloc[]
df = df.dropna(how="all", subset=["amount", "date"])
df = df.where(pd.notnull(df), None)


transaction_dic = {
    t[0]: [t[0], t[1].togregorian().replace(microsecond=0)]
    for t in Transaction.objects.values_list("tracking_code", "transaction_date")
}

last_date = None
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    amount = row["amount"]
    if amount is None or str(amount) in ["nan", "Nan", "NaN"]:
        continue
    date = row["date"]
    if date is not None and date not in ["nan", "Nan", "NaN"]:
        date = jdatetime.strptime(str(date).replace("/", ""), "%Y%m%d").replace(hour=12)
        date = timezone.make_aware(date)
        last_date = date
    else:
        date = last_date
    amount = float(amount)
    course = row["course"]
    course_number = int(row["course_number"]) if pd.notnull(row["course_number"]) else None
    course = get_course(course, course_number)
    person = row["person"]
    account = get_account(row["account"])
    description = row["description"]
    category = get_category(row["category"]) or 9
    transaction_type = 1
    if amount < 0:
        amount = -amount
        transaction_type = 2
    amount = int(amount * 1000 * 1000)
    person = f"person: {person}" if person else None
    if transaction_dic.get(f"FFile-{index}", None) is None:
        Transaction.objects.create(
            transaction_category=category,
            transaction_date=date,
            course=course,
            transaction_type=transaction_type,
            amount=amount,
            account=account,
            tracking_code=f"FFile-{index}",
            description=f"{person if person is not None else ''}\n{description}\nFrom Financial File".strip(),
            fee=0,
        )
        transaction_dic[f"FFile-{index}"] = [f"FFile-{index}", date.togregorian().replace(microsecond=0)]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [02:44<00:00,  3.79it/s]


In [ ]:

from financials.utils import get_payping_transactions
from commons.utils import normalize_phone
from tqdm import tqdm
from financials.utils import make_payping_transaction
from datetime import datetime

payping = FinancialAccount.objects.get(name="پی‌پینگ")
payping_data = get_payping_transactions(datetime(2025, 5, 31, 16, 0, 0), datetime(2025, 7, 21, 0, 0, 10), transaction_type=6)

transaction_dic = {
    t[0]: [t[0], t[1].togregorian().replace(microsecond=0)]
    for t in Transaction.objects.filter(account=payping).values_list("tracking_code", "transaction_date")
}
created_transaction = 0
for res in tqdm(payping_data):
    res["invoiceNo"] = f'PP-{res["invoiceNo"]}'
    if res["description"] != "باشگاه امکان" and make_payping_transaction(res, fee=4500, payping_account=payping, transaction_type=1, transaction_category=None1, transaction_dic=transaction_dic):
        created_transaction += 1
created_transaction

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:12<00:00,  3.60it/s]


42

In [1]:
from financials.utils import get_payping_transactions
from datetime import datetime

payping_data = get_payping_transactions(datetime(2025, 5, 31, 16, 0, 0), datetime(2025, 7, 21, 0, 0, 10), transaction_type=7)
payping_data

[{'reqDate': '2025-07-19T18:30:00',
  'code': '091a24e',
  'isRepaid': True,
  'repayDate': '2025-07-19T20:32:17.148271',
  'amount': 7993400,
  'shaba': 'IR220780850111040707076531',
  'isRefund': False,
  'refundedCode': None,
  'description': None,
  'withdrawType': 'تسویه ساعت 11'},
 {'reqDate': '2025-07-19T18:30:00',
  'code': '836d1f8',
  'isRepaid': True,
  'repayDate': '2025-07-19T20:32:17.126715',
  'amount': 3989200,
  'shaba': 'IR220780850111040707076531',
  'isRefund': False,
  'refundedCode': None,
  'description': None,
  'withdrawType': 'تسویه ساعت 11'},
 {'reqDate': '2025-07-16T18:30:00',
  'code': '4e46e44',
  'isRepaid': True,
  'repayDate': '2025-07-16T20:32:15.355098',
  'amount': 34178160,
  'shaba': 'IR220780850111040707076531',
  'isRefund': False,
  'refundedCode': None,
  'description': None,
  'withdrawType': 'تسویه ساعت 11'},
 {'reqDate': '2025-07-15T18:30:00',
  'code': '7fb420e',
  'isRepaid': True,
  'repayDate': '2025-07-15T20:32:18.543814',
  'amount': 1

In [1]:
from users.models import SMSLogManager, SMSLine
lines = SMSLogManager().get_sms_lines()
# SMSLine.objects.create(provider=0, line_number="9998884289")
lines["9998884289"].send("تست خط اعلانک ۲ با لینک \n https://t.me/neshaa_club", "+989195258134")


In [9]:
import requests
from django.conf import settings

line_number = "9998884289"
recieve_number = "+989195258134"
text = "سلام\nاین یک پیام تست است\n https://neshaa.ir\nلغو۱۱"
# res = requests.get(f"http://payammatni.com/webservice/url/send.php?method=getcredit&format=json&username={settings.ELANAK_USERNAME}&password={settings.ELANAK_PASSWORD}").json()
url = f"https://payammatni.com/webservice/url/send.php?method=sendsms&format=json&from={line_number}&to={recieve_number}&text={text}&type=0&username={settings.ELANAK_USERNAME}&password={settings.ELANAK_PASSWORD}"
res = requests.get(url)